### <청소년, 어린이 Top20 하차 정류장 분석>

In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import seaborn as sns
import platform

In [2]:
# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

In [3]:
# 데이터 불러오기 및 합치기
clear_df=pd.DataFrame()
for i in range(1,81,1):
    file_path ='./data/trfcard ({}).csv'.format(i) 
    df = pd.read_csv(file_path)
    clear_df = pd.concat([clear_df,df],ignore_index=True)
clear_df.head(1)

,on_date,off_date,route_name,descr,age_type,trans_yn,addfee_yn,start_bstop,start_gps_x,start_gps_y,end_bstop,end_gps_x,end_gps_y
0,20200102051049,20200102051844,131,양덕-송도-시청-양학-양덕,일반,N,N,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


In [4]:
# 영문컬럼명 -> 한글컬럼명 변환
col = pd.read_excel('./data/trfcard_columns.xlsx',header=2)
col = col[['컬럼명 (영문)','컬럼명 (한글)']]

col_list = col.to_dict('records')

new_dict = {}

for kv in col_list:
    new_dict[kv['컬럼명 (영문)']] = kv['컬럼명 (한글)']

clear_df = clear_df.rename(columns= new_dict)

In [5]:
# 데이터 전처리(결측치, 이상치)
clear_df.info()
clear_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842608 entries, 0 to 842607
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   승차시각         842608 non-null  int64  
 1   하차시각         842608 non-null  int64  
 2   노선명          842608 non-null  object 
 3   노선설명         842608 non-null  object 
 4   승객연령         842608 non-null  object 
 5   환승여부         842608 non-null  object 
 6   추가운임여부       842608 non-null  object 
 7   승차정류장        842608 non-null  object 
 8   승차정류장 GPS X  842608 non-null  float64
 9   승차정류장 GPS Y  842608 non-null  float64
 10  하차정류장        842608 non-null  object 
 11  하차정류장 GPS X  842608 non-null  float64
 12  하차정류장 GPS Y  842608 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 83.6+ MB


,승차시각,하차시각,승차정류장 GPS X,승차정류장 GPS Y,하차정류장 GPS X,하차정류장 GPS Y
count,8.426080e+05,8.426080e+05,842608.000000,842608.000000,842608.000000,842608.000000
mean,2.020017e+13,2.020017e+13,129.369875,36.033987,129.369699,36.030874
std,6.691819e+07,6.691830e+07,0.051007,0.044272,0.042348,0.034512
min,2.020010e+13,2.020010e+13,129.025585,35.804119,129.025585,35.804119
25%,2.020012e+13,2.020012e+13,129.350290,36.010240,129.353856,36.013250
50%,2.020013e+13,2.020013e+13,129.365486,36.031170,129.365281,36.033732
75%,2.020022e+13,2.020022e+13,129.384770,36.064088,129.371682,36.045735
max,2.020033e+13,2.020040e+13,129.579045,36.303479,129.579045,36.303479


In [6]:
# 데이터 전처리(중복치)
clear_df[clear_df.duplicated()]

,승차시각,하차시각,노선명,노선설명,승객연령,환승여부,추가운임여부,승차정류장,승차정류장 GPS X,승차정류장 GPS Y,하차정류장,하차정류장 GPS X,하차정류장 GPS Y


In [7]:
# 필요 컬럼 불러오기
df_age = clear_df[['노선명','승객연령','승차정류장','승차정류장 GPS X','승차정류장 GPS Y','하차정류장','하차정류장 GPS X','하차정류장 GPS Y']]
df_age.head(1)

,노선명,승객연령,승차정류장,승차정류장 GPS X,승차정류장 GPS Y,하차정류장,하차정류장 GPS X,하차정류장 GPS Y
0,131,일반,양덕차고지,129.401693,36.077258,동부초등학교,129.380866,36.062209


In [279]:
# 청소년, 어린이 구분
df_Teen = df_age[df_age['승객연령'] == '청소년']
df_Child = df_age[df_age['승객연령'] == '어린이']
df_Teen = df_Teen.drop_duplicates()
df_Child = df_Child.drop_duplicates()

In [280]:
# 청소년이 주로 하차하는 상위 20개 하차 정류장 갯수 확인을 위한
# pivot table 생성
df_Teen['탑승순위'] = df_Teen['승객연령']
df_Teen_top20 = df_Teen.pivot_table(index='하차정류장',
                                    values='노선명',
                                    columns='탑승순위',
                                    aggfunc='count').sort_values('청소년',ascending=False).head(20).reset_index()
df_Teen_top20

탑승순위,하차정류장,청소년
0,죽도시장,604
1,시외버스터미널,488
2,중앙상가,352
3,육거리,286
4,홈플러스,236
5,교보생명,234
6,북구청,230
7,선린병원,227
8,북부시장,203
9,남구보건소,197


In [281]:
# 어린이가 주로 하차하는 상위 20개 하차 정류장 갯수 확인을 위한
# pivot table 생성
df_Child['탑승순위'] = df_Child['승객연령']
df_Child_top20 = df_Child.pivot_table(index='하차정류장',
                                      values='노선명',
                                      columns='탑승순위',
                                      aggfunc='count').sort_values('어린이',ascending=False).head(20).reset_index()
df_Child_top20

탑승순위,하차정류장,어린이
0,죽도시장,313
1,시외버스터미널,210
2,중앙상가,148
3,육거리,108
4,남구보건소,106
5,북구청,91
6,선린병원,83
7,교보생명,71
8,홈플러스,69
9,우현경로당,66


In [282]:
# 컬럼명 재 정렬
df_Teen_top20['승객연령'] = '청소년'
df_Child_top20['승객연령'] = '어린이'
df_Teen_top20 = df_Teen_top20.rename(columns = {'청소년':'탑승횟수'})
df_Child_top20 = df_Child_top20.rename(columns = {'어린이':'탑승횟수'})

In [283]:
df_Teen_top20

탑승순위,하차정류장,탑승횟수,승객연령
0,죽도시장,604,청소년
1,시외버스터미널,488,청소년
2,중앙상가,352,청소년
3,육거리,286,청소년
4,홈플러스,236,청소년
5,교보생명,234,청소년
6,북구청,230,청소년
7,선린병원,227,청소년
8,북부시장,203,청소년
9,남구보건소,197,청소년


In [284]:
df_Child_top20

탑승순위,하차정류장,탑승횟수,승객연령
0,죽도시장,313,어린이
1,시외버스터미널,210,어린이
2,중앙상가,148,어린이
3,육거리,108,어린이
4,남구보건소,106,어린이
5,북구청,91,어린이
6,선린병원,83,어린이
7,교보생명,71,어린이
8,홈플러스,69,어린이
9,우현경로당,66,어린이


In [285]:
df_top20_merge = pd.merge(df_Teen_top20, df_Child_top20,how='outer').dropna(axis=1).reset_index(drop=True)
df_top20_merge

탑승순위,하차정류장,탑승횟수,승객연령
0,죽도시장,604,청소년
1,시외버스터미널,488,청소년
2,중앙상가,352,청소년
3,육거리,286,청소년
4,홈플러스,236,청소년
5,교보생명,234,청소년
6,북구청,230,청소년
7,선린병원,227,청소년
8,북부시장,203,청소년
9,남구보건소,197,청소년


In [286]:
# 중복되지 않는 하차 정류장 확인
df_top20_non_dup = df_top20_merge.drop_duplicates(['하차정류장'],keep=False)
df_top20_non_dup

탑승순위,하차정류장,탑승횟수,승객연령
16,GS슈퍼마켓,124,청소년
17,고용복지플러스센터,121,청소년
18,고속버스터미널,120,청소년
19,죽도파출소,118,청소년
33,대동우방아파트,55,어린이
36,장성초등학교,46,어린이
37,유성여고,43,어린이
39,환호해맞이그린빌,42,어린이


In [287]:
# 중복되는 하차 정류장 확인
df_top20_dup = df_top20_merge[df_top20_merge.duplicated(['하차정류장'],keep=False)].reset_index(drop=True)
df_top20_dup

탑승순위,하차정류장,탑승횟수,승객연령
0,죽도시장,604,청소년
1,시외버스터미널,488,청소년
2,중앙상가,352,청소년
3,육거리,286,청소년
4,홈플러스,236,청소년
5,교보생명,234,청소년
6,북구청,230,청소년
7,선린병원,227,청소년
8,북부시장,203,청소년
9,남구보건소,197,청소년


In [289]:
# 중복되는 하차 정류장의 탑승횟수 확인 후 20대이하로 설정
df_top20_dup_sum = df_top20_dup.groupby('하차정류장').sum().sort_values('탑승횟수',ascending=False).reset_index()
df_top20_dup_sum['승객연령'] = '20대이하'
df_top20_dup_sum

탑승순위,하차정류장,탑승횟수,승객연령
0,죽도시장,917,20대이하
1,시외버스터미널,698,20대이하
2,중앙상가,500,20대이하
3,육거리,394,20대이하
4,북구청,321,20대이하
5,선린병원,310,20대이하
6,교보생명,305,20대이하
7,홈플러스,305,20대이하
8,남구보건소,303,20대이하
9,북부시장,267,20대이하


In [320]:
# 청소년,어린이 상위 20 종합
df_top20 = pd.concat([df_top20_dup_sum, df_top20_non_dup])
df_top20

탑승순위,하차정류장,탑승횟수,승객연령
0,죽도시장,917,20대이하
1,시외버스터미널,698,20대이하
2,중앙상가,500,20대이하
3,육거리,394,20대이하
4,북구청,321,20대이하
5,선린병원,310,20대이하
6,교보생명,305,20대이하
7,홈플러스,305,20대이하
8,남구보건소,303,20대이하
9,북부시장,267,20대이하


In [325]:
# 하차정류장 데이터 컬럼 불러오기
df_out = clear_df[['하차정류장','하차정류장 GPS X','하차정류장 GPS Y']].drop_duplicates(['하차정류장'],keep='last')
df_out

,하차정류장,하차정류장 GPS X,하차정류장 GPS Y
1838,진세미골,129.296876,36.038544
9252,원각사,129.402450,35.916136
47324,유계리 교회,129.304767,36.194178
62610,죽천초등학교,129.427028,36.098312
90249,초곡전원타운,129.327282,36.086294
...,...,...,...
842603,농협하나로클럽,129.398558,36.081653
842604,대잠중앙하이츠,129.338901,36.021808
842605,상대시장,129.361983,36.019438
842606,부영사랑3차,129.402238,35.973022


In [ ]:
new_df = pd.merge(left = trfcard_total_freq_2,
            right = trfcard_total_off2,
            how = 'inner',
            left_on='하차정류장',
            right_on = '하차정류장')

In [327]:
# 청소년이 주로 하차하는 상위 20개 하차 정류장
df_top20_final = pd.merge(left = df_top20,right = df_out,left_on='하차정류장',right_on = '하차정류장').reset_index(drop=True)
df_top20_final

,하차정류장,탑승횟수,승객연령,하차정류장 GPS X,하차정류장 GPS Y
0,죽도시장,917,20대이하,129.365169,36.035704
1,시외버스터미널,698,20대이하,129.350580,36.013250
2,중앙상가,500,20대이하,129.365281,36.037455
3,육거리,394,20대이하,129.366269,36.039589
4,북구청,321,20대이하,129.367128,36.041941
5,선린병원,310,20대이하,129.367635,36.047810
6,교보생명,305,20대이하,129.352235,36.015721
7,홈플러스,305,20대이하,129.364714,36.031262
8,남구보건소,303,20대이하,129.397000,35.992908
9,북부시장,267,20대이하,129.368033,36.045153


In [328]:
Pohang_map = folium.Map(
    # 초기 지도의 중앙점 좌표[위도, 경도]
    location=[36.019166, 129.343295],

    # 지도 스타일
    tiles='openstreetmap',
    # 초기 지도의 스케일
    zoom_start= 12
)

for i in range(len(df_top20_final)):
    lat = df_top20_final['하차정류장 GPS Y'].iloc[i]
    lng = df_top20_final['하차정류장 GPS X'].iloc[i]
    type = df_top20_final['승객연령'].iloc[i]
    color = 'black'
    if type == '청소년':
        fillcolor = 'red'
        size = 5
    elif type == '어린이':
        fillcolor = 'yellow'
        size = 5  
    else:
        fillcolor = 'blue'
        size = 5

    # 지도에 표시할 마킹 모양 등 스타일 지정
    folium.CircleMarker(
        location= [lat, lng],
        # 마커 채우기 색상
        fill_color = fillcolor,
        # 마커 채우기 불투명도
        fill_opacity = 1,
        fill=True,
        # 마커 테두리 색상
        color = color,
        # 마커 테두리 굵기
        weight =1,
        # 마커 지름
        radius= size
    ).add_to(Pohang_map)

Pohang_map